# Skip Reindexing and Reuse Saved Embeddings


In this notebook, we will skip the embedding generation and re-indexing process by loading previously saved document embeddings and metadata from a JSON file. We will then use this data to directly query Elasticsearch and the RetrievalQA chain.
    

In [10]:
import json
import numpy as np
from elasticsearch import Elasticsearch
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
import os


# Load the saved embeddings and metadata
with open('document_embeddings.json', 'r') as f:
    saved_data = json.load(f)

# Extract texts, embeddings, and metadata
texts = [item['text'] for item in saved_data]
embeddings = [np.array(item['embedding']) for item in saved_data]
metadatas = [item['metadata'] for item in saved_data]

print("Loaded embeddings and metadata from JSON.")
    

Loaded embeddings and metadata from JSON.


In [6]:

# Initialize Elasticsearch connection
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])
index_name = 'fcra_chunks'

# Check if the index already exists
if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists in Elasticsearch. No re-indexing required.")
else:
    print(f"Index '{index_name}' does not exist. Re-indexing is required.")
    # Add code for reindexing if needed (optional)


Index 'fcra_chunks' already exists in Elasticsearch. No re-indexing required.


In [11]:
#  Load environment variables
load_dotenv()

# Initialize the embedding model with the API key
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_15191/3451141637.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


In [13]:
# Create a Custom Retriever Using Elasticsearch

from langchain.schema import Document
from langchain.schema import BaseRetriever
from typing import Any, List
import numpy as np
from pydantic import BaseModel

class ElasticSearchRetriever(BaseRetriever, BaseModel):
    es: Any
    index_name: str
    embedding_model: Any
    k: int = 5

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Generate and normalize the query embedding
        query_embedding = self.embedding_model.embed_query(query)
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        # Build the script score query
        script_query = {
            "size": self.k,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {
                            "query_vector": query_embedding.tolist()
                        }
                    }
                }
            },
            "_source": ["text", "metadata"]
        }

        # Execute the search
        response = self.es.search(index=self.index_name, body=script_query)

        # Convert hits to Documents
        docs = []
        for hit in response['hits']['hits']:
            doc = Document(
                page_content=hit['_source']['text'],
                metadata=hit['_source']['metadata']
            )
            docs.append(doc)
        return docs

# Initialize the Retriever
retriever = ElasticSearchRetriever(
    es=es,
    index_name=index_name,
    embedding_model=embedding_model,
    k=5  # Number of documents to retrieve
)

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_15191/4229264796.py:9: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ElasticSearchRetriever(BaseRetriever, BaseModel):


In [14]:

# Initialize the LLM (GPT)
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # You can change to map_reduce, etc.
    retriever=retriever
)

# Run a test query
query = "What are the key provisions of the FCRA?"
answer = qa_chain.run(query)
print("Answer:", answer)
    

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_15191/3825728049.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_15191/3825728049.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer:  
The key provisions of the FCRA include:
1. Providing consumers with rights to access and correct data about them held by Consumer Reporting Agencies (CRAs)
2. Imposing obligations on CRAs to protect consumer data and adhere to permissible purpose requirements
3. Defining the class of data that is regulated under the law as information used for determining consumer eligibility for household credit, insurance, employment, or any other permissible purpose
4. Requiring proper disposal of records by entities covered under the law, such as the Federal Trade Commission and other federal agencies.


In [17]:
queries = [
    "What are the permissible purposes for obtaining a consumer report?",
    "Explain the dispute process under the FCRA. Which module did you used to answer this question?",
    "What obligations do credit reporting agencies have according to the FCRA? Which module did you used to answer this question?"
]

for query in queries:
    answer = qa_chain.run(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print("------")

Query: What are the permissible purposes for obtaining a consumer report?
Answer:  The permissible purposes for obtaining a consumer report include determining eligibility for credit, insurance, government benefits, renting an apartment, opening a bank account, or cashing a check. Additionally, a consumer may also initiate a transaction or have an account with an entity that will use the consumer report for a legitimate business need. Other purposes are not considered permissible and include law enforcement, marketing, or similar reasons.
------
Query: Explain the dispute process under the FCRA. Which module did you used to answer this question?
Answer:  I don't know which module specifically discusses the dispute process under the FCRA, but it is mentioned throughout the course. The dispute process under the FCRA allows consumers to dispute any inaccurate or incomplete information in their consumer reports with the consumer reporting agency. The agency must then investigate the disput